In [1]:
import pandas as pd
import polars as pl
import altair as alt

In [2]:
weather_url = 'https://cdn.jsdelivr.net/npm/vega-datasets@1/data/weather.csv'

In [3]:
weather_pl = (
    pl.read_csv(weather_url)
)

In [4]:
weather_pl.head()

location,date,precipitation,temp_max,temp_min,wind,weather
str,str,f64,f64,f64,f64,str
"""Seattle""","""2012-01-01""",0.0,12.8,5.0,4.7,"""drizzle"""
"""Seattle""","""2012-01-02""",10.9,10.6,2.8,4.5,"""rain"""
"""Seattle""","""2012-01-03""",0.8,11.7,7.2,2.3,"""rain"""
"""Seattle""","""2012-01-04""",20.3,12.2,5.6,4.7,"""rain"""
"""Seattle""","""2012-01-05""",1.3,8.9,2.8,6.1,"""rain"""


In [18]:
tempMinMax = alt.Chart(weather_pl).mark_area(opacity=0.3).encode(
    alt.X('month(date):T', title=None, axis=alt.Axis(format='%b')),
    alt.Y('average(temp_max):Q', title='Avg. Temperature °C'),
    alt.Y2('average(temp_min):Q'),
    alt.Color('location:N')
)

tempMinMax

alt.Chart(...)

In [24]:
tempMid = alt.Chart(weather_pl).mark_line().transform_calculate(
    temp_mid='(+datum.temp_min + +datum.temp_max) / 2'
).encode(
    alt.X('month(date):T'),
    alt.Y('average(temp_mid):Q'),
    alt.Color('location:N')
)

In [28]:
tempMinMax + tempMid

alt.LayerChart(...)

In [29]:
alt.layer(tempMid, tempMinMax)

alt.LayerChart(...)

In [31]:
precip = alt.Chart(weather_pl).transform_filter(
    'datum.location == "Seattle"'
).mark_line(
    interpolate='monotone',
    stroke='grey'
).encode(
    alt.X('month(date):T', title=None),
    alt.Y('average(precipitation):Q', title='Precipitation')
)

precip

alt.Chart(...)

In [32]:
tempMinMax_seattle = tempMinMax.transform_filter(
    'datum.location == "Seattle"'
)

In [34]:
alt.layer(tempMinMax_seattle, precip).resolve_scale(y='independent')

alt.LayerChart(...)

In [37]:
alt.Chart(weather_pl).mark_bar().transform_filter(
    'datum.location == "Seattle"'
).encode(
    alt.X('temp_max:Q', bin=True, title='Temperature (°C)'),
    alt.Y('count():Q')
)

alt.Chart(...)

In [40]:
colors = alt.Scale(
    domain=['drizzle', 'fog', 'rain', 'snow', 'sun'],
    range=['#aec7e8', '#c7c7c7', '#1f77b4', '#9467bd', '#e7ba52']
)

alt.Chart(weather_pl).mark_bar().transform_filter(
    'datum.location == "Seattle"'
).encode(
    alt.X('temp_max:Q', bin=True, title='Temperature (°C)'),
    alt.Y('count():Q'),
    alt.Color('weather:N', scale=colors),
    alt.Column('weather:N')
).properties(
    width=150,
    height=150
)

alt.Chart(...)

In [41]:
tempMinMax = alt.Chart().mark_area(opacity=0.3).encode(
  alt.X('month(date):T', title=None, axis=alt.Axis(format='%b')),
  alt.Y('average(temp_max):Q', title='Avg. Temperature (°C)'),
  alt.Y2('average(temp_min):Q'),
  alt.Color('location:N')
)

tempMid = alt.Chart().mark_line().transform_calculate(
  temp_mid='(+datum.temp_min + +datum.temp_max) / 2'
).encode(
  alt.X('month(date):T'),
  alt.Y('average(temp_mid):Q'),
  alt.Color('location:N')
)

alt.layer(tempMinMax, tempMid).facet(
  data=weather_pl,
  column='location:N'
)

alt.FacetChart(...)

In [45]:
base = alt.Chart(weather_pl).mark_line().encode(
  alt.X('month(date):T', title=None),
  color='location:N'
).properties(
  width=240,
  height=180
)

temp = base.encode(alt.Y('average(temp_max):Q'))
precip = base.encode(alt.Y('average(precipitation):Q'))
wind = base.encode(alt.Y('average(wind):Q'))

(temp | precip) & wind.properties(width=480)

alt.VConcatChart(...)